Step 1: Loading Data
For this tutorial, we’ll use the dataset of papers published in NIPS conference. The NIPS conference (Neural Information Processing Systems) is one of the most prestigious yearly events in the machine learning community. The CSV data file contains information on the different NIPS papers that were published from 1987 until 2016 (29 years!). These papers discuss a wide variety of topics in machine learning, from neural networks to optimization methods, and many more.

Let’s start by looking at the content of the file

In [2]:
import zipfile
import pandas as pd
import os

# Open the zip file
with zipfile.ZipFile("./NIPS Papers.zip", "r") as zip_ref:
    # Extract the file to a temporary directory
    zip_ref.extractall("temp")

# Read the CSV file into a pandas DataFrame
papers = pd.read_csv("temp/NIPS Papers/papers.csv")

# Print head
papers.head()

,id,year,title,event_type,pdf_name,abstract,paper_text
0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing,Bayesian Query Construction for Neural\nNetwor...
4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing,"Neural Network Ensembles, Cross\nValidation, a..."


Step 2: Data Cleaning
Since the goal of this analysis is to perform topic modeling, we will solely focus on the text data from each paper, and drop other metadata columns

In [3]:
# Remove the columns
papers = papers.drop(columns=['id', 'title', 'abstract', 
                              'event_type', 'pdf_name', 'year'], axis=1)

# sample only 100 papers
papers = papers.sample(100)

# Print out the first rows of papers
papers.head()

,paper_text
2389,Simplified Rules and Theoretical Analysis for\...
6024,Using hippocampal 'place cells' for\nnavigatio...
1932,From Batch to Transductive Online Learning\n\n...
1603,Information Bottleneck for\nGaussian Variables...
282,A Constructive RBF Network\nfor Writer Adaptat...


Remove punctuation/lower casing
Next, let’s perform a simple preprocessing on the content of paper_text column to make them more amenable for analysis, and reliable results. To do that, we’ll use a regular expression to remove any punctuation, and then lowercase the text

In [4]:
# Load the regular expression library
import re

# Remove punctuation
papers['paper_text_processed'] = papers['paper_text'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
papers['paper_text_processed'] = papers['paper_text_processed'].map(lambda x: x.lower())

# Print out the first rows of papers
papers['paper_text_processed'].head()

2389    simplified rules and theoretical analysis for\...
6024    using hippocampal 'place cells' for\nnavigatio...
1932    from batch to transductive online learning\n\n...
1603    information bottleneck for\ngaussian variables...
282     a constructive rbf network\nfor writer adaptat...
Name: paper_text_processed, dtype: object

Tokenize words and further clean-up text
Let’s tokenize each sentence into a list of words, removing punctuations and unnecessary characters altogether.

In [7]:
import gensim
from gensim.utils import simple_preprocess

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = papers.paper_text_processed.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30])

['simplified', 'rules', 'and', 'theoretical', 'analysis', 'for', 'information', 'bottleneck', 'optimization', 'and', 'pca', 'with', 'spiking', 'neurons', 'lars', 'buesing', 'wolfgang', 'maass', 'institute', 'for', 'theoretical', 'computer', 'science', 'graz', 'university', 'of', 'technology', 'graz', 'austria', 'larsmaass']


Step 3: Phrase Modeling: Bigram and Trigram Models
Bigrams are two words frequently occurring together in the document. Trigrams are 3 words frequently occurring. Some examples in our example are: 'back_bumper', 'oil_leakage', 'maryland_college_park' etc.

Gensim's Phrases model can build and implement the bigrams, trigrams, quadgrams and more. The two important arguments to Phrases are min_count and threshold.

The higher the values of these param, the harder it is for words to be combined.

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

Remove Stopwords, Make Bigrams and Lemmatize
The phrase models are ready. Let’s define the functions to remove the stopwords, make trigrams and lemmatization and call them sequentially.

In [11]:
# NLTK Stop words
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/damlasenturk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [14]:
import spacy

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30])

['simplify', 'rule', 'theoretical', 'analysis', 'information_bottleneck', 'optimization', 'pca', 'spiking_neuron', 'lar', 'buese', 'technology', 'abstract', 'show', 'suitable', 'assumption', 'primarily', 'linearization', 'simple', 'perspicuous', 'online', 'learning', 'rule', 'optimization', 'spiking_neuron', 'derive', 'rule', 'perform', 'common', 'benchmark', 'task']


Step 4: Data transformation: Corpus and Dictionary
The two main inputs to the LDA topic model are the dictionary(id2word) and the corpus. Let’s create them.

In [15]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 1), (1, 2), (2, 1), (3, 4), (4, 2), (5, 1), (6, 2), (7, 1), (8, 1), (9, 4), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 10), (18, 1), (19, 1), (20, 1), (21, 9), (22, 3), (23, 2), (24, 4), (25, 1), (26, 1), (27, 5), (28, 2), (29, 1)]


Step 5: Base Model
We have everything required to train the base LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well. Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior (we'll use default for the base model).

chunksize controls how many documents are processed at a time in the training algorithm. Increasing chunksize will speed up training, at least as long as the chunk of documents easily fit into memory.

passes controls how often we train the model on the entire corpus (set to 10). Another word for passes might be "epochs". iterations is somewhat technical, but essentially it controls how often we repeat a particular loop over each document. It is important to set the number of "passes" and "iterations" high enough.

In [16]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

The above LDA model is built with 10 different topics where each topic is a combination of keywords and each keyword contributes a certain weightage to the topic.

You can see the keywords for each topic and the weightage(importance) of each keyword using lda_model.print_topics()

In [17]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.025*"model" + 0.012*"use" + 0.011*"network" + 0.010*"graph" + 0.010*"set" '
  '+ 0.008*"learn" + 0.007*"give" + 0.007*"structure" + 0.007*"datum" + '
  '0.007*"distribution"'),
 (1,
  '0.023*"model" + 0.010*"influence" + 0.010*"use" + 0.009*"graph" + '
  '0.009*"parameter" + 0.009*"function" + 0.008*"set" + 0.008*"result" + '
  '0.008*"cluster" + 0.008*"time"'),
 (2,
  '0.009*"method" + 0.009*"feature" + 0.008*"set" + 0.008*"training" + '
  '0.008*"use" + 0.007*"figure" + 0.007*"system" + 0.006*"neural" + '
  '0.006*"learn" + 0.006*"rate"'),
 (3,
  '0.011*"network" + 0.010*"cluster" + 0.010*"game" + 0.009*"player" + '
  '0.008*"noise" + 0.008*"point" + 0.008*"value" + 0.008*"set" + 0.007*"use" + '
  '0.006*"learn"'),
 (4,
  '0.008*"model" + 0.008*"use" + 0.008*"point" + 0.008*"datum" + '
  '0.007*"result" + 0.007*"problem" + 0.007*"cluster" + 0.007*"order" + '
  '0.006*"error" + 0.006*"number"'),
 (5,
  '0.010*"use" + 0.010*"problem" + 0.010*"function" + 0.009*"example" + '
 

Compute Model Perplexity and Coherence Score
Let's calculate the baseline coherence score

In [18]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.2715528592528609


Step 6: Hyperparameter tuning¶
First, let's differentiate between model hyperparameters and model parameters :

Model hyperparameters can be thought of as settings for a machine learning algorithm that are tuned by the data scientist before training. Examples would be the number of trees in the random forest, or in our case, number of topics K

Model parameters can be thought of as what the model learns during training, such as the weights for each word in a given topic.

Now that we have the baseline coherence score for the default LDA model, let's perform a series of sensitivity tests to help determine the following model hyperparameters:

Number of Topics (K)
Dirichlet hyperparameter alpha: Document-Topic Density
Dirichlet hyperparameter beta: Word-Topic Density
We'll perform these tests in sequence, one parameter at a time by keeping others constant and run them over the two difference validation corpus sets. We'll use C_v as our choice of metric for performance comparison

In [19]:
 # supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('./results/lda_tuning_results.csv', index=False)
    pbar.close()


  7%|██▋                                     | 36/540 [17:13<4:01:11, 28.71s/it]

 18%|███████                                 | 96/540 [18:34<1:27:07, 11.77s/it]


 36%|█████████████▉                         | 193/540 [37:44<1:11:35, 12.38s/it]


 54%|██████████████████████                   | 290/540 [58:17<51:04, 12.26s/it]


 72%|███████████████████████████▉           | 387/540 [1:18:35<31:33, 12.37s/it]


 84%|████████████████████████████████▋      | 453/540 [1:32:55<18:46, 12.95s/it]

Step 7: Final Model
Based on external evaluation (Code to be added from Excel based analysis), let's train the final model with parameters yielding highest coherence score

In [ ]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [ ]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

Step 8: Visualize Results

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('./results/ldavis_tuned_'+str(num_topics))

# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, './results/ldavis_tuned_'+ str(num_topics) +'.html')

LDAvis_prepared

Hierarchical LDA

In [ ]:
# Assuming documents is a list of tokenized documents
dictionary = Dictionary(data_lemmatized)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Training the top-level LDA model
top_model = LdaModel(corpus, num_topics=10, id2word=dictionary)

# For each topic, we collect the documents most relevant to that topic
topic_docs = {}
for i, doc in enumerate(corpus):
    topic = max(top_model[doc], key=lambda x: x[1])[0]
    if topic not in topic_docs:
        topic_docs[topic] = []
    topic_docs[topic].append(documents[i])

# Now we train a separate LDA model for each set of documents
sub_models = {}
for topic, docs in topic_docs.items():
    sub_dictionary = Dictionary(docs)
    sub_corpus = [sub_dictionary.doc2bow(doc) for doc in docs]
    sub_model = LdaModel(sub_corpus, num_topics=5, id2word=sub_dictionary)
    sub_models[topic] = sub_model
    
    # Visualize the topics
    vis = gensimvis.prepare(sub_model, sub_corpus, sub_dictionary)
    pyLDAvis.display(vis)


Correlated Topic Models

In [ ]:
# initialize the CTM
ctm = CTM(input_vectorizer="tfidf", n_components=10, model_type="prodLDA")

# fit the CTM to the data
ctm.fit(data_lemmatized)

# visualize
topics = ctm.get_topic_lists(5)  # get top 5 words for each topic
topic_word_distributions = ctm.get_topic_word_matrix()
vis_data = gensimvis.prepare(topic_word_distributions, df['processed'], ctm.vectorizer)
pyLDAvis.display(vis_data)